#### Static Attributes Completness Ratio

In [1]:
import pandas as pd

c:\Users\n9810200\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\n9810200\AppData\Local\Temp/ipykernel_321476/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
truck_df_dirigo = pd.read_csv('../Dirigo/OCEL_Truck.csv')
cargo_df_dirigo = pd.read_csv('../Dirigo/OCEL_Cargo.csv')

In [3]:
truck_object_ids = truck_df_dirigo["Object_id"].unique()
cargo_object_ids = cargo_df_dirigo["Object_id"].unique()

In [4]:
truck_static_attrs = ["LPT", "Axles"]
cargo_static_attrs = ["Cargo Type", "Silo ID"]

In [ ]:
# truck_total_pairs = len(truck_object_ids) * len(truck_static_attrs)
# cargo_total_pairs = len(cargo_object_ids) * len(cargo_static_attrs)
# denominator = truck_total_pairs + cargo_total_pairs
# denominator

140

##### Dirigo

In [ ]:
def identify_static_attributes(df, skip_cols=None):
    if skip_cols is None:
        skip_cols = ["Timestamp", "Ocel_changed_field"]
    static_attrs = []
    for attr in df.columns:
        if attr == "Object_id" or attr in skip_cols:
            continue
        unique_counts = df.groupby("Object_id")[attr].nunique(dropna=True)
        if (unique_counts <= 1).all():
            static_attrs.append(attr)
    return static_attrs

In [6]:
def compute_static_completeness_numerator(df,static_attrs):
    result = {}
    for attr in static_attrs:
        filled_count = df.dropna(subset=[attr]).groupby("Object_id")[attr].nunique().sum()
        result[attr] = filled_count
    return result

In [8]:
truck_static_attrs = identify_static_attributes(truck_df_dirigo)
# -- ['LPT', 'Axles']
cargo_static_attrs = identify_static_attributes(cargo_df_dirigo)
# -- ['Cargo Type', 'Silo ID']

In [17]:
truck_filled_total = compute_static_completeness_numerator(truck_df_dirigo, truck_static_attrs)

cargo_filled_total = compute_static_completeness_numerator(cargo_df_dirigo, cargo_static_attrs)


In [18]:
num_cargo_objects = cargo_df_dirigo['Object_id'].nunique()

cargo_attr_ratios = {
    attr: filled / num_cargo_objects
    for attr, filled in cargo_filled_total.items()
}
print("Cargo attribute completeness ratios:")
for attr, ratio in cargo_attr_ratios.items():
    print(f"{attr}: {ratio:.2%}")

Cargo attribute completeness ratios:
Cargo Type: 100.00%
Silo ID: 100.00%


In [20]:
num_truck_objects = truck_df_dirigo['Object_id'].nunique()

truck_attr_ratios = {
    attr: filled / num_truck_objects
    for attr, filled in truck_filled_total.items()
}
print("Truck attribute completeness ratios:")
for attr, ratio in truck_attr_ratios.items():
    print(f"{attr}: {ratio:.2%}")

Truck attribute completeness ratios:
LPT: 100.00%
Axles: 100.00%


##### ACEL

In [22]:
staticAttri_df = pd.read_csv('../ACEL/ACEL_Objects.csv')

In [33]:
attribute_map_by_type = {}

object_types = staticAttri_df['Object Type'].unique()

for ot in object_types:
    df_sub = staticAttri_df[staticAttri_df['Object Type'] == ot]
    attr_cols = [
        col for col in staticAttri_df.columns
        if col not in ["ObjectId", "Object Type"] and df_sub[col].notna().any()
    ]
    attribute_map_by_type[ot] = attr_cols

attribute_map_by_type


{'Cargo': ['Cargo Type', 'Silo ID'], 'Truck': ['LPT', 'Axles']}

In [36]:
object_counts = staticAttri_df.groupby("Object Type")["ObjectId"].nunique().to_dict()

attribute_map_by_type = {}
object_types = staticAttri_df['Object Type'].unique()

for ot in object_types:
    df_sub = staticAttri_df[staticAttri_df['Object Type'] == ot]
    attr_cols = [
        col for col in staticAttri_df.columns
        if col not in ["ObjectId", "Object Type"] and df_sub[col].notna().any()
    ]
    attribute_map_by_type[ot] = attr_cols


print("Static Attribute Completeness Ratios:\n")

for ot, attrs in attribute_map_by_type.items():
    df_sub = staticAttri_df[staticAttri_df['Object Type'] == ot]
    num_objects = object_counts[ot]

    print(f"Object Type: {ot}")
    for attr in attrs:
        filled_count = df_sub[attr].notna().sum()
        ratio = filled_count / num_objects
        print(f"{attr}: {ratio:.2%}")
    print()


Static Attribute Completeness Ratios:

Object Type: Cargo
Cargo Type: 100.00%
Silo ID: 100.00%

Object Type: Truck
LPT: 100.00%
Axles: 100.00%



##### DOCEL

In [38]:
truckStatic_df = pd.read_csv('../DOCEL/DOCEL_Truck_Static.csv')
cargoStatic_df = pd.read_csv('../DOCEL/DOCEL_Cargo_Static.csv')

In [39]:
truckattr_cols = [col for col in truckStatic_df.columns if col not in ["Truck ID"]]
cargoattr_cols = [col for col in cargoStatic_df.columns if col not in ["Cargo ID"]]


In [51]:
truckAttr_counts_per_attr = truckStatic_df[truckattr_cols].notna().sum()
truckAttr_counts_per_attr


LPT      50
Axles    50
dtype: int64

In [50]:
cargoAttr_counts_per_attr = cargoStatic_df[cargoattr_cols].notna().sum()
cargoAttr_counts_per_attr

Cargo Type    20
Silo ID       20
dtype: int64

In [52]:
lpt_ratio = truckAttr_counts_per_attr['LPT']/ len(truckStatic_df['Truck ID'])
axles_ratio = truckAttr_counts_per_attr['Axles']/ len(truckStatic_df['Truck ID'])
cargoType_ratio = cargoAttr_counts_per_attr['Cargo Type']/ len(cargoStatic_df['Cargo ID'])
cargoLoc_ratio = cargoAttr_counts_per_attr['Silo ID']/ len(cargoStatic_df['Cargo ID'])

In [55]:
print("Static Attribute Completeness Ratio for DOCEL:")

print("Truck")
print("LPT: {:.2f}%".format(lpt_ratio * 100))
print("Axles: {:.2f}%".format(axles_ratio * 100))

print("Cargo")
print("Cargo Type: {:.2f}%".format(cargoType_ratio * 100))
print("Silo ID: {:.2f}%".format(cargoLoc_ratio * 100))


Static Attribute Completeness Ratio for DOCEL:
Truck
LPT: 100.00%
Axles: 100.00%
Cargo
Cargo Type: 100.00%
Silo ID: 100.00%
